
# Катастрофическое забывание

## Цель: Проверить влияние fine-tuning на исходную модель.

## Описание/Пошаговая инструкция выполнения домашнего задания:
1. Скачать датасет ImageNette: https://github.com/fastai/imagenette (`ImageNette` это подвыборка из 10 классов датасета `ImageNet`).
2. Взять предобученную на обычном `ImageNet` модель `ResNet18` и заменить число классов на 10.
3. Дообучить модель на 10 классах `ImageNette` и замерить точность (эта точность будет считаться базовой). Обучить только последний слой. Сохранить последний слой как оригинальный.
4. Дообучить модель классифицировать датасет `CIFAR10`.
5. Вернуть оригинальный последний слой модели и проверить качество на `ImageNette` и сравнить с базовой точностью.
6. Дообучить только последний слой (отключить градиент для всех слоев кроме последнего) на `ImageNette` и проверить удалось ли добиться исходного качества.
7. Сделать выводы.

### Критерии оценки:
__Принято__ - задание выполнено полностью.

__Возвращено на доработку__ - задание не выполнено полностью.


In [108]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm

## Шаг 1 Подготовка датасета

### 1.1 Определение настроек

In [136]:
TRAIN_DIR_IMAGENETTE = 'DataForModel/imagenette2-320/train'
VAL_DIR_IMAGENETTE = 'DataForModel/imagenette2-320/val'
TRAIN_DIR_CIFAR10 = 'DataForModel/cifar10/train'
VAL_DIR_CIFAR10 = 'DataForModel/cifar10/val'
BATCH_SIZE = 32
NUM_EPOCHS = 5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

### 1.2 Датасет ImageNette

In [137]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset_imagenette = torchvision.datasets.ImageFolder(TRAIN_DIR_IMAGENETTE, transform=transform)
val_dataset_imagenette = torchvision.datasets.ImageFolder(VAL_DIR_IMAGENETTE, transform=transform)

imagenette_train = torch.utils.data.DataLoader(train_dataset_imagenette, batch_size=BATCH_SIZE, shuffle=True)
imagenette_val = torch.utils.data.DataLoader(val_dataset_imagenette, batch_size=BATCH_SIZE)

### 1.3 Датасет CIFAR10

In [ ]:
train_dataset_cifar = torchvision.datasets.CIFAR10(root=TRAIN_DIR_CIFAR10, train=True, download=True, transform=transform)
val_dataset_cifar = torchvision.datasets.CIFAR10(root=VAL_DIR_CIFAR10, train=False, download=True, transform=transform)

cifar_train = torch.utils.data.DataLoader(train_dataset_cifar, BATCH_SIZE, shuffle=True), 
cifar_val = torch.utils.data.DataLoader(val_dataset_cifar, BATCH_SIZE)

## Шаг 2: Замена последнего слоя на 10 классов

In [139]:
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# Заменяем последний слой на 10 классов
model.fc = nn.Linear(model.fc.in_features, 10)
model.fc.requires_grad = True # Разморозка последнего слоя
model = model.to(DEVICE)

## Шаг 3: Обучение на ImageNette (последний слой)

### Шаг 3.1 Обучение модели

In [140]:
# Расчет точности
def evaluate(model, val_loader):
    model.eval() 
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.0001)
base_acc = 0.0

for epoch in range(NUM_EPOCHS):
    # Обучение
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(imagenette_train, desc=f'Epoch {epoch+1}'):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Валидация
    acc = evaluate(model, imagenette_val)
    if acc > base_acc:
        base_acc = acc
        torch.save(model.fc.state_dict(), 'imagenette_fc.pth')

print(f"Базовая точность: {base_acc:.2f}%")

## Шаг 4: Обучение на CIFAR10

In [ ]:
cifar_acc = 0.0

for epoch in range(NUM_EPOCHS):
    # Обучение
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(cifar_train, desc=f'Epoch {epoch+1}'):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Валидация
    acc = evaluate(model, cifar_val)
    if acc > cifar_acc:
        cifar_acc = acc

print(f"CIFAR точность: {cifar_acc:.2f}%")


## Шаг 5: Возврат оригинального слоя и проверка качества на ImageNette

### 5.1 Возврат оригинального слоя

In [ ]:

model.fc.load_state_dict(torch.load('imagenette_fc.pth'))

### 5.2 Проверка качестка на ImageNette

In [ ]:
new_acc = evaluate(model, imagenette_val)
print(f"Базовая точность: {base_acc:.2f}%")
print(f"\nТочность с оригинальным слоем ImageNette: {new_acc:.2f}%")


## Шаг 6: Дообучить последний слой на ImageNette и проверить удалось ли добиться исходного качества.

### 6.1 Заморозка последнего слоя

In [ ]:
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True

### 6.2 Обучение на ImageNette

In [ ]:
last_acc = 0.0

for epoch in range(NUM_EPOCHS):
    # Обучение
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(imagenette_train, desc=f'Epoch {epoch+1}'):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Валидация
    acc = evaluate(model, imagenette_val)
    if acc > last_acc:
        last_acc = acc

print(f"Попытка вернуть точность: {cifar_acc:.2f}%")

# Вывод:

Текст вывода